In [2]:
from tensorflow import keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os

In [47]:
batch_size = 50
num_classes = 10
epochs = 32
data_augmentation = True
num_predictions = 20
model_name = 'keras_cifar10_trained_model.h5'
save_dir = os.path.join(os.getcwd(), 'tmp/keras_data')

In [48]:
os.getcwd(), save_dir

('/Users/alexandr/Учеба/Мага/giis/lab3',
 '/Users/alexandr/Учеба/Мага/giis/lab3/tmp/keras_data')

In [49]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [50]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [51]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [52]:
opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
      rotation_range = 20,
      width_shift_range = 0.2,
      height_shift_range = 0.2,
      horizontal_flip = True
    )

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=1)

Using real-time data augmentation.
Epoch 1/32
1000/1000 [==============================] - 42s 42ms/step - loss: 2.0096 - acc: 0.2645 - val_loss: 1.7552 - val_acc: 0.3997
Epoch 2/32
1000/1000 [==============================] - 44s 44ms/step - loss: 1.7759 - acc: 0.3554 - val_loss: 1.5730 - val_acc: 0.4498
Epoch 3/32
1000/1000 [==============================] - 46s 46ms/step - loss: 1.6688 - acc: 0.3928 - val_loss: 1.5122 - val_acc: 0.4604
Epoch 4/32
1000/1000 [==============================] - 53s 53ms/step - loss: 1.6105 - acc: 0.4160 - val_loss: 1.4323 - val_acc: 0.4954
Epoch 5/32
1000/1000 [==============================] - 51s 51ms/step - loss: 1.5681 - acc: 0.4323 - val_loss: 1.4101 - val_acc: 0.4965
Epoch 6/32
1000/1000 [==============================] - 52s 52ms/step - loss: 1.5321 - acc: 0.4477 - val_loss: 1.3838 - val_acc: 0.5000
Epoch 7/32
1000/1000 [==============================] - 52s 52ms/step - loss: 1.5034 - acc: 0.4579 - val_loss: 1.3770 - val_acc: 0.5010
Epoch 8/32
10

Epoch 15/32
1000/1000 [==============================] - 51s 51ms/step - loss: 1.3384 - acc: 0.5212 - val_loss: 1.1914 - val_acc: 0.5831
Epoch 16/32
1000/1000 [==============================] - 49s 49ms/step - loss: 1.3308 - acc: 0.5237 - val_loss: 1.2002 - val_acc: 0.5820
Epoch 17/32
1000/1000 [==============================] - 55s 55ms/step - loss: 1.3187 - acc: 0.5311 - val_loss: 1.1482 - val_acc: 0.5926
Epoch 18/32
1000/1000 [==============================] - 51s 51ms/step - loss: 1.3127 - acc: 0.5330 - val_loss: 1.1566 - val_acc: 0.5876
Epoch 19/32
1000/1000 [==============================] - 60s 60ms/step - loss: 1.2961 - acc: 0.5389 - val_loss: 1.1639 - val_acc: 0.5819
Epoch 20/32
1000/1000 [==============================] - 54s 54ms/step - loss: 1.2927 - acc: 0.5392 - val_loss: 1.1291 - val_acc: 0.6035
Epoch 21/32
1000/1000 [==============================] - 53s 53ms/step - loss: 1.2836 - acc: 0.5435 - val_loss: 1.1581 - val_acc: 0.5954
Epoch 22/32
1000/1000 [==================

KeyboardInterrupt: 

In [40]:
# model_path = os.path.join(save_dir, model_name)
# model.save(model_path)
model.save(save_dir)
print('Saved trained model at {}'.format(save_dir))

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /Users/alexandr/Учеба/Мага/giis/lab3/tmp/keras_data
10000/10000 [==============================] - 2s 194us/sample - loss: 1.3353 - acc: 0.5214
Test loss: 1.3352680932998657
Test accuracy: 0.5214
